In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import json
from datetime import date
from dataclasses import dataclass, field, asdict
from typing import List, Optional
pio.renderers.default = "notebook"

In [2]:
df = pd.read_parquet('../data/player_final_data.parquet', engine = 'fastparquet')

In [3]:
df2 = pd.read_parquet('../data/player_final_data_season_level.parquet', engine = 'fastparquet')

In [4]:
transfermarkt_data = pd.read_parquet('../data/transfermarkt_player_data_final_v2.parquet', engine = 'fastparquet')

In [5]:
season_pre = ['2013/2014', '2014/2015', '2015/2016', '2016/2017', '2017/2018', '2018/2019']
season_post = ['2019/2020', '2020/2021', '2021/2022', '2022/2023', '2023/2024', '2024/2025']
season_post1 = ['2019/2020', '2020/2021', '2021/2022']
season_post2 = ['2022/2023', '2023/2024', '2024/2025']

In [6]:
from utils import scrap_data

In [7]:
clubs = scrap_data.SEASONS_IN_EKSTRAKLASA

In [ ]:
df[(df['Sezon'] == '2013/2014') & (df['Status Młodzieżowca'] == True)][['Klub', 'czas']].groupby(['Klub']).sum('czas').sort_values(['czas'], ascending = False)

In [ ]:
df[(df['Sezon'] == '2013/2014') & (df['Status Młodzieżowca'] == True) & (df['Klub'] == 'Lechia Gdańsk')]

In [ ]:
df[(df['Sezon'] == '2013/2014') & (df['Status Młodzieżowca'] == True) & (df['Klub'] == 'Widzew Łódź')]

### funkcje

In [ ]:
def bar_chart_1_y_axis(title, subtitle, x_axis, x_axis_title, y_axis, y_axis_title):
    fig = go.Figure(data=go.Bar(
        x=x_axis,
        y=y_axis,
        marker_color='#7A7A7A',  # Change bar color
        text=y_axis,              # Data labels
        textposition='auto',             # 'auto', 'outside', 'inside', etc.
    ))
    
    # Customize layout
    fig.update_layout(
        title={
            'text': f'{title}<br><sub>{subtitle}</sub>',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        title_font_size=22,
        xaxis_title=x_axis_title,
        yaxis_title=y_axis_title,
        xaxis=dict(
            tickfont_size=12,
            title_font=dict(size=14),
            showgrid=False,
        ),
        font=dict(family="Inter, Arial, sans-serif"),
        yaxis=dict(
            tickfont_size=12,
            title_font=dict(size=14),
            showgrid=True,
            gridcolor='lightgray'
        ),
        paper_bgcolor="#FFFDF8",
        plot_bgcolor="#FFFDF8",    # Area around the chart
        margin=dict(t=80, l=50, r=30, b=50)
    )
    
    return fig

In [ ]:
def bar_chart_2_y_axis(title, subtitle, x_axis, x_axis_title, y_primary_axis, y_primary_axis_title, y_secondary_axis, y_secondary_axis_title):
    fig = go.Figure(
        data=[
            go.Bar(name=y_primary_axis_title, x=x_axis, y=y_primary_axis, yaxis='y', text=y_primary_axis, textposition='auto', offsetgroup=1, marker_color="#1f77b4"),        
            go.Bar(name=y_secondary_axis_title, x=x_axis, y=y_secondary_axis, yaxis='y2', text=y_secondary_axis, textposition='auto', offsetgroup=2, marker_color="#ff7f0e")
        ],
        layout={
            'yaxis': {'title': y_primary_axis_title},
            'yaxis2': {'title': y_secondary_axis_title, 'overlaying': 'y', 'side': 'right'}
        }
    )
    
    # Change the bar mode
    fig.update_layout(
                title={
                'text': f'{title}<br><sub>{subtitle}</sub>',
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            legend=dict(
                yanchor="top",
                orientation="h",
                y=0.99,
                xanchor="left",
                x=0.003
            ),
            font=dict(family="Inter, Arial, sans-serif"),
            title_font_size=22,
            barmode='group',
            xaxis_title=x_axis_title,
            xaxis=dict(
                tickfont_size=12,
                title_font=dict(size=14),
                showgrid=False,
            ),
            paper_bgcolor="#FFFDF8",
            plot_bgcolor="#FFFDF8",    # Area around the chart
            margin=dict(t=80, l=50, r=30, b=50))

    return fig

### Ekstraklasa w liczbach

#### Liczba meczów w trakcie sezonu

In [ ]:
sezony = df[['Sezon', 'w.']].groupby(['Sezon']).max(['w.'])

In [ ]:
sezony

In [ ]:
bar_chart_1_y_axis('Liczba meczów na sezon'
                   , 'Liczba rozegranych meczów w Ekstraklasie w analizowanych sezonach.'
                   , sezony.index
                   , 'Sezon'
                   , sezony['w.']
                   , 'Liczba meczów')

#### liczba sezonów spędzonych w Eklapie przed/po zmianie

In [ ]:
clubs_season_data = df[df['Przepis Aktywny'] == 'w trakcie'][['Klub', 'Sezon']].drop_duplicates()
clubs_season_data_agg_during = pd.DataFrame(clubs_season_data.groupby(['Klub']).size())
clubs_season_data_agg_during = clubs_season_data_agg_during.rename(columns={0: "sezony_w_trakcie_przepisu"})

In [ ]:
clubs_season_data = df[df['Przepis Aktywny'] == 'przed'][['Klub', 'Sezon']].drop_duplicates()
clubs_season_data_agg_pre = pd.DataFrame(clubs_season_data.groupby(['Klub']).size())
clubs_season_data_agg_pre = clubs_season_data_agg_pre.rename(columns={0: "sezony_przed_przepisem"})

In [ ]:
clubs_season_data = df[['Klub', 'Sezon']].drop_duplicates()
clubs_season_data_agg = pd.DataFrame(clubs_season_data.groupby(['Klub']).size())
clubs_season_data_agg = clubs_season_data_agg.rename(columns={0: "sezony_ogolem"})

In [ ]:
clubs_season_data_agg = clubs_season_data_agg.join([clubs_season_data_agg_during, clubs_season_data_agg_pre], how = 'left', validate = 'one_to_one')
clubs_season_data_agg = clubs_season_data_agg.fillna(0)
clubs_season_data_agg['sezony_w_trakcie_przepisu'] = clubs_season_data_agg['sezony_w_trakcie_przepisu'].apply(lambda x : int(x))
clubs_season_data_agg['sezony_przed_przepisem'] = clubs_season_data_agg['sezony_przed_przepisem'].apply(lambda x : int(x))
clubs_season_data_agg = clubs_season_data_agg.sort_values(['sezony_ogolem', 'sezony_w_trakcie_przepisu'], ascending = False)

In [ ]:
clubs_season_data_agg

### sezony klubów w Ekstraklasie

In [ ]:
club_season = df[['Klub', 'Sezon']].drop_duplicates()

In [ ]:
club_dict = {}

In [ ]:
for idx in club_season.index:
    if not club_season.at[idx, 'Klub'] in club_dict.keys():
        club_dict[club_season.at[idx, 'Klub']] = [club_season.at[idx, 'Sezon']]
    else:
        club_dict[club_season.at[idx, 'Klub']].append(club_season.at[idx, 'Sezon'])

In [ ]:
club_dict

### Młodzieżowcy w liczbach

#### Liczba młodzieżowców na sezon

In [ ]:
#wszyscy zawodnicy
all_players_num = df[['Sezon', 'ID']].drop_duplicates().groupby(['Sezon']).size()

In [ ]:
#młodzieżowcy
u22_players_num = df[df['Status Młodzieżowca'] == True][['Sezon', 'ID']].drop_duplicates().groupby(['Sezon']).size()

In [ ]:
ratio1 = u22_players_num/all_players_num

In [ ]:
u22_players_num

In [ ]:
ratio1

In [ ]:
all_players_num

### Czas gry młodzieżowców

#### Czas gry młodzieżowców na sezon

In [ ]:
player_all_sum_play_time_season = df.groupby(['Sezon'])['czas'].sum().reset_index(name='sum_time_all')
player_all_sum_play_time_season['sum_time_all'] = player_all_sum_play_time_season['sum_time_all'].apply(lambda x : int(x))
player_u22_sum_play_time_season = df[df['Status Młodzieżowca'] == True].groupby(['Sezon'])['czas'].sum().reset_index(name='sum')
player_u22_sum_play_time_season['sum'] = player_u22_sum_play_time_season['sum'].apply(lambda x : int(x))
player_u22_sum_play_time_season = player_u22_sum_play_time_season.join(player_all_sum_play_time_season, rsuffix = '_all')[['Sezon', 'sum', 'sum_time_all']]
player_u22_sum_play_time_season['u22_pct'] = round(player_u22_sum_play_time_season['sum']/player_u22_sum_play_time_season['sum_time_all'], 2)*100
player_u22_sum_play_time_season['u22_pct'] = player_u22_sum_play_time_season['u22_pct'].apply(lambda x : int(x))

player_u22_sum_play_time_season_bar_chart = bar_chart_1_y_axis('Udział młodzieżowców w czasie gry',
                                                           'Ile z możliwego dostępnego czasu gry było udziałem młodzieżowców?',
                                                           player_u22_sum_play_time_season['Sezon'].to_list(),
                                                           'Sezon',
                                                            player_u22_sum_play_time_season['u22_pct'].to_list(),
                                                           'Udział w czasie gry [%]')

In [ ]:
player_u22_sum_play_time_season

In [ ]:
player_u22_sum_play_time_season_bar_chart.show()

#### Szczegółowy czas gry młodzieżowców

In [ ]:
df2[df2['Status Młodzieżowca'] == True][['Sezon', 'Poniżej 900 minut', 'Powyżej 900 minut', 'Pełne 10 meczów', 'Zmieniony 10 meczów', 'Wszedł z ławki 10 meczow', 'Powyżej 1620 minut', 'Pełne 18 meczów', 'Zmieniony 18 meczów', 'Wszedł z ławki 18 meczow']].groupby(['Sezon'])[['Poniżej 900 minut', 'Powyżej 900 minut', 'Pełne 10 meczów', 'Zmieniony 10 meczów', 'Wszedł z ławki 10 meczow', 'Powyżej 1620 minut', 'Pełne 18 meczów', 'Zmieniony 18 meczów', 'Wszedł z ławki 18 meczow']].agg(['sum', 'mean'])

In [ ]:
df2[(df2['Status Młodzieżowca'] == True) & (df2['Sezon'].isin(season_pre))][['Poniżej 900 minut', 'Powyżej 900 minut', 'Pełne 10 meczów', 'Zmieniony 10 meczów', 'Wszedł z ławki 10 meczow', 'Powyżej 1620 minut', 'Pełne 18 meczów', 'Zmieniony 18 meczów', 'Wszedł z ławki 18 meczow']].agg(['sum', 'mean'])

In [ ]:
df2[(df2['Status Młodzieżowca'] == True) & (df2['Sezon'].isin(season_post))][['Poniżej 900 minut', 'Powyżej 900 minut', 'Pełne 10 meczów', 'Zmieniony 10 meczów', 'Wszedł z ławki 10 meczow', 'Powyżej 1620 minut', 'Pełne 18 meczów', 'Zmieniony 18 meczów', 'Wszedł z ławki 18 meczow']].agg(['sum', 'mean'])

In [ ]:
df2[(df2['Status Młodzieżowca'] == True) & (df2['Sezon'].isin(season_post1))][['Poniżej 900 minut', 'Powyżej 900 minut', 'Pełne 10 meczów', 'Zmieniony 10 meczów', 'Wszedł z ławki 10 meczow', 'Powyżej 1620 minut', 'Pełne 18 meczów', 'Zmieniony 18 meczów', 'Wszedł z ławki 18 meczow']].agg(['sum', 'mean'])

In [ ]:
df2[(df2['Status Młodzieżowca'] == True) & (df2['Sezon'].isin(season_post2))][['Poniżej 900 minut', 'Powyżej 900 minut', 'Pełne 10 meczów', 'Zmieniony 10 meczów', 'Wszedł z ławki 10 meczow', 'Powyżej 1620 minut', 'Pełne 18 meczów', 'Zmieniony 18 meczów', 'Wszedł z ławki 18 meczow']].agg(['sum', 'mean'])

In [ ]:
# Example data
data1 = df2[(df2['Status Młodzieżowca'] == True) & (df2['Sezon'].isin(season_pre))]['czas']
data2 = df2[(df2['Status Młodzieżowca'] == True) & (df2['Sezon'].isin(season_post))]['czas']
df_hst = pd.DataFrame({
    "values": list(data1) + list(data2),
    "group": ["przed zmianą"] * len(data1) + ["po zmianie"] * len(data2)
})

In [ ]:
fig = px.histogram(
    df_hst,
    x="values",
    color="group",
    barmode="overlay",   # or "group"
    opacity=0.6,
    histnorm="percent",
    nbins=100
)

fig.show()

### Młodzieżowcy w liczbach - nieużyte statystyki

In [ ]:
player_count_under_22above_1620_min = df[df['Powyżej 1620 minut'] == True].groupby(['Sezon', 'Status Młodzieżowca']).size().reset_index(name='counts')
player_count_under_22above_1620_min = player_count_under_22above_1620_min[player_count_under_22above_1620_min['Status Młodzieżowca'] == True][['Sezon', 'counts']].reset_index(drop = True)

In [ ]:
player_count_under_22above_1620_min_all = df[df['Powyżej 1620 minut'] == True].groupby(['Sezon']).size().reset_index(name='counts_all')
player_count_under_22above_1620_min_all = player_count_under_22above_1620_min_all.join(player_count_under_22above_1620_min, rsuffix = '_u22')[['Sezon', 'counts_all', 'counts']]
player_count_under_22above_1620_min_all['u22_pct'] = round(player_count_under_22above_1620_min_all['counts']/player_count_under_22above_1620_min_all['counts_all'], 2)*100
player_count_under_22above_1620_min_all['u22_pct'] = player_count_under_22above_1620_min_all['u22_pct'].apply(lambda x : int(x))

In [ ]:
player_count_under_22_bar_chart = bar_chart_2_y_axis(
                                            'Liczba młodzieżowców którzy zagrali przynajmniej 1620 minut na sezon',
                                            'Ilu młodzieżowców wystąpiło w danym sezonie? | Jaki % z wszystkich zawodników którzy wystąpili w danym sezonie stanowili młodzieżowcy?',
                                            player_count_under_22above_1620_min['Sezon'].to_list(),
                                            'Sezon',
                                            player_count_under_22above_1620_min['counts'].to_list(),
                                            'Liczba młodzieżowców [#]',
                                            player_count_under_22above_1620_min_all['u22_pct'].to_list(),
                                            'Procent młodzieżowców [%]')

In [ ]:
player_count_under_22_bar_chart.show()

In [ ]:
player_u22_average_play_time_season = df[df['Status Młodzieżowca'] == True].groupby(['Sezon'])['czas'].mean().reset_index(name='average')
player_u22_average_play_time_season['average'] = player_u22_average_play_time_season['average'].apply(lambda x : int(x))
player_u22_average_play_time_season_bar_chart = bar_chart(player_u22_average_play_time_season['Sezon'].to_list(),
                                                           player_u22_average_play_time_season['average'].to_list(),
                                                           'Średni czas gry młodzieżowców na sezon',
                                                           'Każdy który zagrał przynajmniej minutę',
                                                           'Sezon',
                                                           'Średni czas gry [min]')

In [ ]:
player_u22_average_play_time_season_bar_chart.show()

In [ ]:
player_u22_median_play_time_season = df[df['Status Młodzieżowca'] == True].groupby(['Sezon'])['czas'].median().reset_index(name='median')
player_u22_median_play_time_season['median'] = player_u22_median_play_time_season['median'].apply(lambda x : int(x))
player_u22_median_play_time_season_bar_chart = bar_chart(player_u22_median_play_time_season['Sezon'].to_list(),
                                                           player_u22_median_play_time_season['median'].to_list(),
                                                           'Mediana czasu gry młodzieżowców w sezonie',
                                                           'Każdy który zagrał przynajmniej minutę',
                                                           'Sezon',
                                                           'Mediana czasu gry [min]')

In [ ]:
player_u22_median_play_time_season_bar_chart.show()

In [ ]:
player_u22_sum_play_time_season

In [ ]:
player_u22_avg_play_time_period = df[df['Status Młodzieżowca'] == True].groupby(['Przepis Aktywny'])['czas'].mean().reset_index(name='avg')
player_u22_avg_play_time_period['avg'] = player_u22_avg_play_time_period['avg'].apply(lambda x : int(x))
player_u22_avg_play_time_period_bar_chart = bar_chart(player_u22_avg_play_time_period['Przepis Aktywny'].to_list(),
                                                           player_u22_avg_play_time_period['avg'].to_list(),
                                                           'Średni czas gry młodzieżowców w okresie',
                                                           'Każdy który zagrał przynajmniej minutę. ',
                                                           'Okres przed/w trakcie trwania przepisu',
                                                           'Średni czas gry [min]')

In [ ]:
player_u22_avg_play_time_period_bar_chart.show()

In [ ]:
player_u22_median_play_time_period = df[df['Status Młodzieżowca'] == True].groupby(['Przepis Aktywny'])['czas'].median().reset_index(name='median')
player_u22_median_play_time_period['median'] = player_u22_median_play_time_period['median'].apply(lambda x : int(x))
player_u22_median_play_time_period_bar_chart = bar_chart(player_u22_median_play_time_period['Przepis Aktywny'].to_list(),
                                                           player_u22_median_play_time_period['median'].to_list(),
                                                           'Mediana czasu gry młodzieżowców w okresie',
                                                           'Każdy który zagrał przynajmniej minutę. ',
                                                           'Okres przed/w trakcie trwania przepisu',
                                                           'Mediana czasu gry [min]')

In [ ]:
player_u22_median_play_time_period_bar_chart.show()

In [ ]:
player_non_u22_sum_play_time_period = df[df['Status Młodzieżowca'] == False].groupby(['Przepis Aktywny'])['czas'].sum().reset_index(name='sum_non_u22')
player_non_u22_sum_play_time_period['sum_non_u22'] = player_non_u22_sum_play_time_period['sum_non_u22'].apply(lambda x : int(x))
player_u22_sum_play_time_period = df[df['Status Młodzieżowca'] == True].groupby(['Przepis Aktywny'])['czas'].sum().reset_index(name='sum')
player_u22_sum_play_time_period['sum'] = player_u22_sum_play_time_period['sum'].apply(lambda x : int(x))
player_u22_sum_play_time_period = player_u22_sum_play_time_period.join(player_non_u22_sum_play_time_period, rsuffix = '_non_u22')[['Przepis Aktywny', 'sum', 'sum_non_u22']]
player_u22_sum_play_time_period['u22_pct'] = round(player_u22_sum_play_time_period['sum']/player_u22_sum_play_time_period['sum_non_u22'], 2)*100
player_u22_sum_play_time_period['u22_pct'] = player_u22_sum_play_time_period['u22_pct'].apply(lambda x : int(x))

player_u22_sum_play_time_period_bar_chart = bar_chart_1_y_axis(
                                                               'Udział młodzieżowców w czasie gry',
                                                           'Ile z możliwego dostępnego czasu gry było udziałem młodzieżowców?',
                                                            player_u22_sum_play_time_period['Przepis Aktywny'].to_list(),
                                                           'Okres przed/w trakcie trwania przepisu',
                                                            player_u22_sum_play_time_period['u22_pct'].to_list(),
                                                           'Udział w czasie gry [%]')

In [ ]:
player_u22_sum_play_time_period_bar_chart.show()

### Młodzieżowcy a kluby

#### czas gry młodzieżowców

In [ ]:
clubs_player_sum_all_pre = df[df['Przepis Aktywny'] == 'przed'][['Klub', 'czas', 'Status Młodzieżowca']].groupby(['Klub', 'Status Młodzieżowca']).sum(['czas'])

In [ ]:
clubs_player_sum_all_pre

In [ ]:
clubs_player_sum_all_during = df[df['Przepis Aktywny'] == 'w trakcie'][['Klub', 'czas', 'Status Młodzieżowca']].groupby(['Klub', 'Status Młodzieżowca']).sum(['czas'])

In [ ]:
clubs_player_sum_all_during

In [ ]:
clubs_player_sum_all_during_mandatory = df[df['Sezon'].isin(season_post1)][['Klub', 'czas', 'Status Młodzieżowca']].groupby(['Klub', 'Status Młodzieżowca']).sum(['czas'])

In [ ]:
clubs_player_sum_all_during_mandatory

In [ ]:
clubs_player_sum_all_during_optional = df[df['Sezon'].isin(season_post2)][['Klub', 'czas', 'Status Młodzieżowca']].groupby(['Klub', 'Status Młodzieżowca']).sum(['czas'])

In [ ]:
clubs_player_sum_all_during_optional

#### liczba młodieżowców

In [ ]:
clubs_player_count_all_pre = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Przepis Aktywny'] == 'przed')].groupby(['Klub']).size())

In [ ]:
clubs_player_count_all_post = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Przepis Aktywny'] == 'w trakcie')].groupby(['Klub']).size())

In [ ]:
clubs_player_count_all_during_mandatory = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Sezon'].isin(season_post1))].groupby(['Klub']).size())

In [ ]:
clubs_player_count_all_during_optional = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Sezon'].isin(season_post2))].groupby(['Klub']).size())

In [ ]:
clubs_player_count_all_during_optional

#### średnia i mediana czasu gry młodzieżowców w klubach

In [ ]:
clubs_player_sum_all_pre_mean_prep = df[(df['Przepis Aktywny'] == 'przed') & (df['Status Młodzieżowca'] == True)][['Klub', 'czas']].groupby(['Klub']).describe()

In [ ]:
clubs_player_sum_all_pre_mean = clubs_player_sum_all_pre_mean_prep.iloc[:, clubs_player_sum_all_pre_mean_prep.columns.get_level_values(1).isin(['count', 'mean', '50%'])]

In [ ]:
clubs_player_sum_all_post_mean_prep = df[(df['Przepis Aktywny'] == 'w trakcie') & (df['Status Młodzieżowca'] == True)][['Klub', 'czas']].groupby(['Klub']).describe()

In [ ]:
clubs_player_sum_all_post_mean = clubs_player_sum_all_post_mean_prep.iloc[:, clubs_player_sum_all_post_mean_prep.columns.get_level_values(1).isin(['count', 'mean', '50%'])]

In [ ]:
clubs_player_sum_all_post_mean_prep_obl = df[(df['Sezon'].isin(season_post1)) & (df['Status Młodzieżowca'] == True)][['Klub', 'czas']].groupby(['Klub']).describe()

In [ ]:
clubs_player_sum_all_pre_mean_obl = clubs_player_sum_all_post_mean_prep_obl.iloc[:, clubs_player_sum_all_post_mean_prep_obl.columns.get_level_values(1).isin(['count', 'mean', '50%'])]

In [ ]:
clubs_player_sum_all_post_mean_prep_opt = df[(df['Sezon'].isin(season_post2)) & (df['Status Młodzieżowca'] == True)][['Klub', 'czas']].groupby(['Klub']).describe()

In [ ]:
clubs_player_sum_all_pre_mean_opt = clubs_player_sum_all_post_mean_prep_opt.iloc[:, clubs_player_sum_all_post_mean_prep_opt.columns.get_level_values(1).isin(['count', 'mean', '50%'])]

In [ ]:
final = pd.concat([clubs_player_sum_all_pre_mean, clubs_player_sum_all_post_mean, clubs_player_sum_all_pre_mean_obl, clubs_player_sum_all_pre_mean_opt], axis=1)

In [ ]:
final

### Liczba zawodników i czas gry po utracie statusu młodzieżowca

In [ ]:
young = df2[(df2['Młodzieżowiec ostatni sezon'] == True) & (df2['Sezon'] != '2024/2025')][['ID', 'Wiek', 'Sezon', 'Imie i Nazwisko', 'w.', 'pełne', 'z ławki', 'czas']] .set_index('ID')

In [ ]:
old = df2[(df2['Senior pierwszy sezon'] == True) & (df2['Sezon'] != '2013/2014')][['ID', 'Wiek', 'Sezon', 'Imie i Nazwisko', 'w.', 'pełne', 'z ławki', 'czas']] .set_index('ID')

In [ ]:
young_old = pd.merge(young, old, how='outer', left_index=True, right_index=True, suffixes=('_young', '_old'))

In [ ]:
young_old

In [ ]:
young_old_is_nan = young_old.isnull()

In [ ]:
for idx in young_old_is_nan.index:
    if young_old_is_nan.at[idx, 'Imie i Nazwisko_young'] == False and young_old_is_nan.at[idx, 'Imie i Nazwisko_old'] == False:
        young_old.at[idx, 'Kategoria'] = 'Młodzieżowiec grał, Senior grał'
    elif young_old_is_nan.at[idx, 'Imie i Nazwisko_young'] == False and young_old_is_nan.at[idx, 'Imie i Nazwisko_old'] == True:
        young_old.at[idx, 'Kategoria'] = 'Młodzieżowiec grał, Senior nie grał'
    elif young_old_is_nan.at[idx, 'Imie i Nazwisko_young'] == True and young_old_is_nan.at[idx, 'Imie i Nazwisko_old'] == False:
        young_old.at[idx, 'Kategoria'] = 'Młodzieżowiec nie grał, Senior grał'
    else:
        young_old.at[idx, 'Kategoria'] = 'Something went wrong'

In [ ]:
young_old.groupby(['Kategoria'])[['czas_young', 'czas_old']].sum()

In [ ]:
young_old.groupby(['Kategoria', 'Sezon_old'])[['czas_young', 'czas_old']].sum()

In [ ]:
young_old.groupby(['Kategoria', 'Sezon_old']).size()

In [ ]:
young_old

### transfery w momencie utraty statusu młodzieżowca

In [ ]:
young = df2[(df2['Młodzieżowiec ostatni sezon'] == True) & (df2['Sezon'] != '2024/2025')][['ID', 'Wiek', 'Sezon', 'Imie i Nazwisko', 'w.', 'pełne', 'z ławki', 'czas']] .set_index('ID')

In [ ]:
old = df2[(df2['Senior pierwszy sezon'] == True) & (df2['Sezon'] != '2013/2014')][['ID', 'Wiek', 'Sezon', 'Imie i Nazwisko', 'w.', 'pełne', 'z ławki', 'czas']] .set_index('ID')

In [ ]:
young_old = pd.merge(young, old, how='outer', left_index=True, right_index=True, suffixes=('_young', '_old'))

In [ ]:
for idx in young_old_is_nan.index:
    if young_old_is_nan.at[idx, 'Imie i Nazwisko_young'] == False and young_old_is_nan.at[idx, 'Imie i Nazwisko_old'] == False:
        young_old.at[idx, 'Kategoria'] = 'Młodzieżowiec grał, Senior grał'
    elif young_old_is_nan.at[idx, 'Imie i Nazwisko_young'] == False and young_old_is_nan.at[idx, 'Imie i Nazwisko_old'] == True:
        young_old.at[idx, 'Kategoria'] = 'Młodzieżowiec grał, Senior nie grał'
    elif young_old_is_nan.at[idx, 'Imie i Nazwisko_young'] == True and young_old_is_nan.at[idx, 'Imie i Nazwisko_old'] == False:
        young_old.at[idx, 'Kategoria'] = 'Młodzieżowiec nie grał, Senior grał'
    else:
        young_old.at[idx, 'Kategoria'] = 'Something went wrong'

In [ ]:
young_old[young_old['Imie i Nazwisko_young'] == 'Michał Nalepa']

In [ ]:
transfermarkt_data.head()

In [ ]:
def merge_transfers_with_categories(t_df, c_df):
    t = t_df.copy(deep = True)
    c = c_df.copy(deep = True)
    for idx in t.index:
        id_90minut = t.at[idx, 'ID 90minut']
        season_young_na_check = c.isna()
        transfer_date = t.at[idx, 'Transfer: data'].split('.')
        if len(transfer_date) < 3:
            print(f"Nieznana data transferu zawodnika: {t.at[idx, 'Imie i Nazwisko']} w sezonie: {t.at[idx, 'Transfer: sezon']} z klubu: {t.at[idx, 'Transfer: z klubu']} do klubu: {t.at[idx, 'Transfer: do klubu']}")
            transfer_year = '0'
            transfer_month = '0'
        else:
            transfer_year = transfer_date[2]
            transfer_month = transfer_date[1]
        
        if id_90minut in c.index and season_young_na_check.at[id_90minut, 'Sezon_young'] == False:
            season_young_date = c.at[id_90minut, 'Sezon_young'].split(r'/')
            season_young_end_year = season_young_date[1]
            if  transfer_year == season_young_end_year:
                season_young = c.at[id_90minut, 'Sezon_young']
                season_old = c.at[id_90minut, 'Sezon_old']
                category = c.at[id_90minut, 'Kategoria']

                t.at[idx, 'Sezon_young'] = season_young
                t.at[idx, 'Sezon_old'] = season_old
                t.at[idx, 'Kategoria'] = category
                t.at[idx, 'Transfer_rok'] = transfer_year
    t = t.dropna(thresh = 12)
    return t

In [ ]:
test = merge_transfers_with_categories(transfermarkt_data, young_old)

In [ ]:
test[~test['Transfer: cena'].isin(['koniec wypożyczenia', 'wypożyczenie'])].shape

In [ ]:
test[~test['Transfer: cena'].isin(['koniec wypożyczenia', 'wypożyczenie'])].head(20)

In [ ]:
test[~test['Transfer: cena'].isin(['koniec wypożyczenia', 'wypożyczenie'])].groupby(['Transfer_rok']).size()

In [ ]:
test[~test['Transfer: cena'].isin(['koniec wypożyczenia', 'wypożyczenie'])].to_csv('transfery_mlodziezowiec_na_seniora.csv', index = False)

### transfery młodzieżowców

In [14]:
def merge_all_transfers_with_categories(t_df, c_df):
    t = t_df.copy(deep = True)
    c = c_df.copy(deep = True)
    for idx in t.index:
        id_90minut = t.at[idx, 'ID 90minut']
        transfer_sezon_all = t.at[idx, 'Transfer: sezon'].split(r'/')
        if len(transfer_sezon_all) < 2:
            print(f"Nieznana data transferu zawodnika: {t.at[idx, 'Imie i Nazwisko']} w sezonie: {t.at[idx, 'Transfer: sezon']} z klubu: {t.at[idx, 'Transfer: z klubu']} do klubu: {t.at[idx, 'Transfer: do klubu']}")
            transfer_sezon = '10000'
        else:
            transfer_sezon = transfer_sezon_all[0]
            season_play_pre_transfer = f'{int(transfer_sezon)-1}/{transfer_sezon}'
        
        # if id_90minut in c.index:
        if type(c.at[id_90minut, 'Sezon'])==str:
            season_young_date = c.at[id_90minut, 'Sezon'].split(r'/')
            season_young_end_year = season_young_date[1]
            if  transfer_sezon <= season_young_end_year and t.at[idx, 'Transfer: z klubu'] in (clubs.keys()) and season_play_pre_transfer in (clubs[t.at[idx, 'Transfer: z klubu']]):
                season_young = c.at[id_90minut, 'Sezon']
                t.at[idx, 'Sezon_young'] = season_young
                if c.at[id_90minut, 'Młodzieżowiec ostatni sezon'] == True and (int(c.at[id_90minut, 'Sezon'].split('/')[0]) < int(t.at[idx, 'Transfer: sezon'].split(r'/')[0])):
                    t.at[idx, 'Transfer po utracie statusu'] = True
                else: 
                    t.at[idx, 'Transfer po utracie statusu'] = False
                print(f"Transferu zawodnika: {t.at[idx, 'Imie i Nazwisko']} w sezonie: {t.at[idx, 'Transfer: sezon']} z klubu: {t.at[idx, 'Transfer: z klubu']}")
        else:
            subdf = c.loc[id_90minut, :].reset_index()
            for ix in subdf.index:
                season_young_date = subdf.at[ix, 'Sezon'].split(r'/')
                season_young_end_year = season_young_date[1]
            if  transfer_sezon <= season_young_end_year and t.at[idx, 'Transfer: z klubu'] in (clubs.keys()) and season_play_pre_transfer in (clubs[t.at[idx, 'Transfer: z klubu']]):
                season_young = subdf.at[ix, 'Sezon']
                t.at[idx, 'Sezon_young'] = season_young
                if subdf.at[ix, 'Młodzieżowiec ostatni sezon'] == True and (int(subdf.at[ix, 'Sezon'].split('/')[0]) < int(t.at[idx, 'Transfer: sezon'].split(r'/')[0])):
                    t.at[idx, 'Transfer po utracie statusu'] = True
                else: 
                    t.at[idx, 'Transfer po utracie statusu'] = False
                print(f"Transferu zawodnika: {t.at[idx, 'Imie i Nazwisko']} w sezonie: {t.at[idx, 'Transfer: sezon']} z klubu: {t.at[idx, 'Transfer: z klubu']}")
    #t = t.dropna(thresh = 12)
    t = t[t['Transfer: z klubu'].isin(clubs.keys())]
    return t[~t['Sezon_young'].isnull()]

In [15]:
young_all = df2[df2['Status Młodzieżowca'] == True][['ID', 'Wiek', 'Sezon', 'Imie i Nazwisko', 'w.', 'pełne', 'z ławki', 'czas', 'Młodzieżowiec ostatni sezon']] .set_index('ID')

In [16]:
df2[df2['Imie i Nazwisko'] == 'Dariusz Sosnowski']

,Sezon,ID,Imie i Nazwisko,w.,pełne,z ławki,czas,Polskie Obywatelstwo,Wiek,Status Młodzieżowca,...,Powyżej 900 minut,Pełne 10 meczów,Zmieniony 10 meczów,Wszedł z ławki 10 meczow,Powyżej 1620 minut,Pełne 18 meczów,Zmieniony 18 meczów,Wszedł z ławki 18 meczow,Młodzieżowiec ostatni sezon,Senior pierwszy sezon
417,2013/2014,24417.0,Dariusz Sosnowski,1,0,1,1,True,19,True,...,False,False,False,False,False,False,False,False,False,False


In [17]:
test_all = merge_all_transfers_with_categories(transfermarkt_data, young_all)

Transferu zawodnika: Michał Chrapek w sezonie: 2014/2015 z klubu: Wisła Kraków
Transferu zawodnika: Dominik Kun w sezonie: 2015/2016 z klubu: Pogoń Szczecin
Transferu zawodnika: Wojciech Lisowski w sezonie: 2014/2015 z klubu: Pogoń Szczecin
Transferu zawodnika: Michal Efir w sezonie: 2014/2015 z klubu: Legia Warszawa
Transferu zawodnika: Mateusz Kupczak w sezonie: 2014/2015 z klubu: Podbeskidzie Bielsko-Biała
Transferu zawodnika: Michal Kopczynski w sezonie: 2014/2015 z klubu: Legia Warszawa
Transferu zawodnika: Jakub Bak w sezonie: 2015/2016 z klubu: Pogoń Szczecin
Transferu zawodnika: Jakub Bak w sezonie: 2014/2015 z klubu: Pogoń Szczecin
Transferu zawodnika: Tomasz Kowalski w sezonie: 2014/2015 z klubu: Widzew Łódź
Transferu zawodnika: Szymon Drewniak w sezonie: 2015/2016 z klubu: Lech Poznań
Transferu zawodnika: Szymon Drewniak w sezonie: 2014/2015 z klubu: Górnik Zabrze
Transferu zawodnika: Mateusz Lewandowski w sezonie: 2014/2015 z klubu: Pogoń Szczecin
Transferu zawodnika: Mateu

In [20]:
test_all.head()

,ID 90minut,ID transfermarkt,Imie i Nazwisko,Pozycja,Transfer: sezon,Transfer: data,Transfer: z klubu,Transfer: do klubu,Transfer: wartość rynkowa,Transfer: cena,Sezon_young,Transfer po utracie statusu
27,13389,117683,Michał Chrapek,Ofensywny pomocnik,2014/2015,01.07.2014,Wisła Kraków,Catania Calcio,1200000,1400000,2013/2014,True
63,14582,201193,Dominik Kun,Defensywny pomocnik,2015/2016,28.08.2015,Pogoń Szczecin,Wisła Płock,200000,25000,2014/2015,True
97,15331,182953,Wojciech Lisowski,Środkowy obrońca,2014/2015,16.07.2014,Pogoń Szczecin,Chojniczanka,250000,za darmo,2013/2014,True
113,15914,169217,Michal Efir,Środkowy napastnik,2014/2015,01.07.2014,Legia Warszawa,Ruch Chorzów,150000,25000,2013/2014,True
143,16442,237523,Mateusz Kupczak,Defensywny pomocnik,2014/2015,01.07.2014,Podbeskidzie Bielsko-Biała,Termalica Bruk-Bet Nieciecza,200000,wypożyczenie,2013/2014,True


In [21]:
test_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 847 entries, 27 to 5951
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID 90minut                   847 non-null    int64 
 1   ID transfermarkt             847 non-null    int64 
 2   Imie i Nazwisko              847 non-null    object
 3   Pozycja                      847 non-null    object
 4   Transfer: sezon              847 non-null    object
 5   Transfer: data               847 non-null    object
 6   Transfer: z klubu            847 non-null    object
 7   Transfer: do klubu           847 non-null    object
 8   Transfer: wartość rynkowa    847 non-null    object
 9   Transfer: cena               847 non-null    object
 10  Sezon_young                  847 non-null    object
 11  Transfer po utracie statusu  847 non-null    object
dtypes: int64(2), object(10)
memory usage: 86.0+ KB


In [ ]:
### dodaj jeszcze flaga na transfer po ostatnim sezonie jako młodzieżowiec i będzie gitówa

In [22]:
test_all[~test_all['Transfer: cena'].isin(['koniec wypożyczenia', 'wypożyczenie'])].to_csv('transfery_mlodziezowiec_wszystkie_v4.csv', index = False)

In [36]:
test_all[test_all['Imie i Nazwisko'] == 'Kelechukwu Ibe-Torti']

,ID 90minut,ID transfermarkt,Imie i Nazwisko,Pozycja,Transfer: sezon,Transfer: data,Transfer: z klubu,Transfer: do klubu,Transfer: wartość rynkowa,Transfer: cena,Sezon_young,Transfer po utracie statusu
5951,41491,607450,Kelechukwu Ibe-Torti,Prawy skrzydłowy,2024/2025,23.01.2025,ŁKS Łódź,Stalowa Wola,75000,za darmo,2023/2024,True


In [35]:
clubs['Wisła Płock']

['2022/2023',
 '2021/2022',
 '2020/2021',
 '2019/2020',
 '2018/2019',
 '2017/2018',
 '2016/2017']

In [8]:
transfermarkt_data.head()

,ID 90minut,ID transfermarkt,Imie i Nazwisko,Pozycja,Transfer: sezon,Transfer: data,Transfer: z klubu,Transfer: do klubu,Transfer: wartość rynkowa,Transfer: cena
0,12027,158798,Piotr Mroziński,Prawy obrońca,2021/2022,15.07.2021,Stalowa Wola,Puszcza Niepołomice,-,za darmo
1,12027,158798,Piotr Mroziński,Prawy obrońca,2018/2019,17.07.2018,Pogon Siedlce,Stalowa Wola,75000,za darmo
2,12027,158798,Piotr Mroziński,Prawy obrońca,2016/2017,04.01.2017,Wisła Płock,Pogon Siedlce,150000,za darmo
3,12027,158798,Piotr Mroziński,Prawy obrońca,2015/2016,06.07.2015,Widzew Łódź,Wisła Płock,200000,za darmo
4,12027,158798,Piotr Mroziński,Prawy obrońca,2012/2013,30.06.2013,Sandecja Nowy Sącz,Widzew Łódź,200000,koniec wypożyczenia


### Inne

In [ ]:
clubs_player_count_above_1620_all = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Powyżej 10 meczów'] == True)].groupby(['Klub']).size())
clubs_player_count_above_1620_all = clubs_player_count_above_1620_all.rename(columns={0: "liczba młodzieżowców ogółem powyżej 10 meczów"})

In [ ]:
clubs_player_count_above_1620_all

In [ ]:
clubs_player_count_above_1620_pre = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Powyżej 10 meczów'] == True) & (df['Przepis Aktywny'] == 'przed')].groupby(['Klub']).size())
clubs_player_count_above_1620_pre = clubs_player_count_above_1620_pre.rename(columns={0: "liczba młodzieżowców przed przepisem powyżej 10 meczów"})

In [ ]:
clubs_player_count_above_1620_pre

In [ ]:
clubs_player_count_above_1620_during = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Powyżej 10 meczów'] == True) & (df['Przepis Aktywny'] == 'w trakcie')].groupby(['Klub']).size())
clubs_player_count_above_1620_during = clubs_player_count_above_1620_during.rename(columns={0: "liczba młodzieżowców w trakcie przepisu powyżej 10 meczów"})

In [ ]:
clubs_player_count_above_1620_during

In [ ]:
clubs_player_avg_play_time_all = pd.DataFrame(df.groupby(['Klub'])[['czas']].sum())
clubs_player_avg_play_time_u22_all = pd.DataFrame(df[df['Status Młodzieżowca'] == True].groupby(['Klub'])['czas'].sum())
clubs_player_avg_play_time_all = clubs_player_avg_play_time_all.join(clubs_player_avg_play_time_u22_all, how = 'left', lsuffix = '_all', rsuffix = '_u22')
clubs_player_avg_play_time_all['procent_gry_u22_ogolem'] = clubs_player_avg_play_time_all['czas_u22']*100.0/clubs_player_avg_play_time_all['czas_all']
clubs_player_avg_play_time_all['procent_gry_u22_ogolem'] = clubs_player_avg_play_time_all['procent_gry_u22_ogolem'].apply(lambda x : round(x, 1))
clubs_player_avg_play_time_all = clubs_player_avg_play_time_all.rename(columns={'czas_u22': "czas_u22_ogolem"})
clubs_player_avg_play_time_all = clubs_player_avg_play_time_all[['czas_u22_ogolem', 'procent_gry_u22_ogolem']]

In [ ]:
clubs_player_avg_play_time_all

In [ ]:
clubs_player_avg_play_time_pre = pd.DataFrame(df[df['Przepis Aktywny'] == 'przed'].groupby(['Klub'])[['czas']].sum())
clubs_player_avg_play_time_u22_pre = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Przepis Aktywny'] == 'przed')].groupby(['Klub'])['czas'].sum())
clubs_player_avg_play_time_pre = clubs_player_avg_play_time_pre.join(clubs_player_avg_play_time_u22_pre, how = 'left', lsuffix = '_all', rsuffix = '_u22')
clubs_player_avg_play_time_pre['procent_gry_u22_przed_przepisem'] = clubs_player_avg_play_time_pre['czas_u22']*100.0/clubs_player_avg_play_time_pre['czas_all']
clubs_player_avg_play_time_pre['procent_gry_u22_przed_przepisem'] = clubs_player_avg_play_time_pre['procent_gry_u22_przed_przepisem'].apply(lambda x : round(x, 1))
clubs_player_avg_play_time_pre = clubs_player_avg_play_time_pre.rename(columns={'czas_u22': "czas_u22_przed_przepisem"})
clubs_player_avg_play_time_pre = clubs_player_avg_play_time_pre[['czas_u22_przed_przepisem', 'procent_gry_u22_przed_przepisem']]

In [ ]:
clubs_player_avg_play_time_pre

In [ ]:
clubs_player_avg_play_time_during = pd.DataFrame(df[df['Przepis Aktywny'] == 'w trakcie'].groupby(['Klub'])[['czas']].sum())
clubs_player_avg_play_time_u22_during = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Przepis Aktywny'] == 'w trakcie')].groupby(['Klub'])['czas'].sum())
clubs_player_avg_play_time_during = clubs_player_avg_play_time_during.join(clubs_player_avg_play_time_u22_during, how = 'left', lsuffix = '_all', rsuffix = '_u22')
clubs_player_avg_play_time_during['procent_gry_u22_w_trakcie_przepisu'] = clubs_player_avg_play_time_during['czas_u22']*100.0/clubs_player_avg_play_time_during['czas_all']
clubs_player_avg_play_time_during['procent_gry_u22_w_trakcie_przepisu'] = clubs_player_avg_play_time_during['procent_gry_u22_w_trakcie_przepisu'].apply(lambda x : round(x, 1))
clubs_player_avg_play_time_during = clubs_player_avg_play_time_during.rename(columns={'czas_u22': "czas_u22_w_trakcie_przepisu"})
clubs_player_avg_play_time_during = clubs_player_avg_play_time_during[['czas_u22_w_trakcie_przepisu', 'procent_gry_u22_w_trakcie_przepisu']]

In [ ]:
clubs_player_avg_play_time_during

In [ ]:
clubs_player_avg_play = pd.DataFrame(df[df['Status Młodzieżowca'] == True].groupby(['Klub'])[['czas']].mean())
clubs_player_avg_play = clubs_player_avg_play.rename(columns={'czas': 'średni czas gry ogółem'})
clubs_player_avg_play['średni czas gry ogółem'] = clubs_player_avg_play['średni czas gry ogółem'].apply(lambda x : int(x))

In [ ]:
clubs_player_avg_play

In [ ]:
clubs_player_avg_play_during = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Przepis Aktywny'] == 'w trakcie')].groupby(['Klub'])[['czas']].mean())
clubs_player_avg_play_during = clubs_player_avg_play_during.rename(columns={'czas': 'średni czas gry w trakcie przepisu'})
clubs_player_avg_play_during['średni czas gry w trakcie przepisu'] = clubs_player_avg_play_during['średni czas gry w trakcie przepisu'].apply(lambda x : int(x))

In [ ]:
clubs_player_avg_play_during

In [ ]:
clubs_player_avg_play_pre = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Przepis Aktywny'] == 'przed')].groupby(['Klub'])[['czas']].mean())
clubs_player_avg_play_pre = clubs_player_avg_play_pre.rename(columns={'czas': 'średni czas gry przed przepisem'})
clubs_player_avg_play_pre['średni czas gry przed przepisem'] = clubs_player_avg_play_pre['średni czas gry przed przepisem'].apply(lambda x : int(x))

In [ ]:
clubs_player_avg_play_pre

In [ ]:
clubs_player_median_play = pd.DataFrame(df[df['Status Młodzieżowca'] == True].groupby(['Klub'])[['czas']].median())
clubs_player_median_play = clubs_player_median_play.rename(columns={'czas': 'mediana czasu gry ogółem'})
clubs_player_median_play['mediana czasu gry ogółem'] = clubs_player_median_play['mediana czasu gry ogółem'].apply(lambda x : int(x))

In [ ]:
clubs_player_median_play

In [ ]:
clubs_player_median_play_during = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Przepis Aktywny'] == 'w trakcie')].groupby(['Klub'])[['czas']].median())
clubs_player_median_play_during = clubs_player_median_play_during.rename(columns={'czas': 'mediana czasu gry w trakcie przepisu'})
clubs_player_median_play_during['mediana czasu gry w trakcie przepisu'] = clubs_player_median_play_during['mediana czasu gry w trakcie przepisu'].apply(lambda x : int(x))

In [ ]:
clubs_player_median_play_during

In [ ]:
clubs_player_median_play_pre = pd.DataFrame(df[(df['Status Młodzieżowca'] == True) & (df['Przepis Aktywny'] == 'przed')].groupby(['Klub'])[['czas']].median())
clubs_player_median_play_pre = clubs_player_median_play_pre.rename(columns={'czas': 'mediana czasu gry przed przepisem'})
clubs_player_median_play_pre['mediana czasu gry przed przepisem'] = clubs_player_median_play_pre['mediana czasu gry przed przepisem'].apply(lambda x : int(x))

In [ ]:
clubs_player_median_play_pre

In [ ]:
clubs_agg = clubs_season_data_agg.join([clubs_player_count_all, clubs_player_count_all_during, clubs_player_count_all_pre, clubs_player_count_above_1620_all, clubs_player_count_above_1620_pre,
                           clubs_player_count_above_1620_during, clubs_player_avg_play_time_all, clubs_player_avg_play_time_pre, clubs_player_avg_play_time_during,
                           clubs_player_avg_play, clubs_player_avg_play_during, clubs_player_avg_play_pre, clubs_player_median_play, clubs_player_median_play_during, 
                           clubs_player_median_play_pre], how = 'left')
clubs_agg['liczba_mlodziezowcow_na_sezon_ogolem'] = clubs_agg['liczba młodzieżowców ogółem']/clubs_agg['sezony_ogolem']
clubs_agg['liczba_mlodziezowcow_10_meczow_na_sezon_ogolem'] = clubs_agg['liczba młodzieżowców ogółem powyżej 10 meczów']/clubs_agg['sezony_ogolem']
clubs_agg['liczba_mlodziezowcow_na_sezon_przed_przepisem'] = clubs_agg['liczba młodzieżowców przed przepisem']/clubs_agg['sezony_przed_przepisem']
clubs_agg['liczba_mlodziezowcow_10_meczow_na_sezon_przed_przepisem'] = clubs_agg['liczba młodzieżowców przed przepisem powyżej 10 meczów']/clubs_agg['sezony_przed_przepisem']
clubs_agg['liczba_mlodziezowcow_na_sezon_w_trakcie_przepisu'] = clubs_agg['liczba młodzieżowców w trakcie trwania przepisu']/clubs_agg['sezony_w_trakcie_przepisu']
clubs_agg['liczba_mlodziezowcow_10_meczow_na_sezon_w_trakcie_przepisu'] = clubs_agg['liczba młodzieżowców w trakcie przepisu powyżej 10 meczów']/clubs_agg['sezony_w_trakcie_przepisu']
clubs_agg

In [ ]:
df.head()

In [ ]:
df[df['Status Młodzieżowca'] == True]['Imie i Nazwisko'].drop_duplicates()